In [323]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [324]:
df = pd.read_csv('task_3a_sample_data.csv')
df.head()

,public_id,title,text,our rating
0,f2182a54,HUGE! Attorney Sidney Powell CONFIRMS Alleged ...,Last week Rep. Louie Gohmert told Chris Salced...,FALSE
1,c5175d8d,Paul Ryan’s Worst Ally - The New York Times,WHATEVER drama plays out when Republicans meet...,TRUE
2,213a870b,"You Can Get Jail Time Or $3,000 Fine For Not W...",Source page URL Title You Can Get Jail Time O...,FALSE
3,392886ea,Antifa gearing up for false flag violence disg...,With merchants in Democrat-run cities boarding...,FALSE
4,bc6d5d55,Remarks by President Biden on the Administrati...,State Dining Room 4:22 P.M. EST THE PRESIDEN...,partially false


In [325]:
X = df.drop('our rating', axis=1)
y = df['our rating']

X['total'] = X['title'] + ' ' + X['text']

X.shape, y.shape

((50, 4), (50,))

In [326]:
words = X.copy()

In [327]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/skyehigh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/skyehigh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [328]:
import re
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
corpus = []
corpus_test = []

for i in range(len(words)):
    review = re.sub('[^a-zA-Z]', ' ', words['total'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    

In [329]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
X = tfidf_vec.fit_transform(corpus).toarray()
X.shape

(50, 5000)

In [330]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [334]:
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

previous_score = 0
for alpha in np.arange(0, 1, 0.1):
    sub_classifier = MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train, y_train)
    y_pred = sub_classifier.predict(X_test)
    score = metrics.accuracy_score(y_test, y_pred)
    
    if score > previous_score:
        classifier = sub_classifier
        previous_score = score


/usr/local/lib/python3.9/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


In [335]:
prediction = classifier.predict(X_test)

report = metrics.classification_report(y_test, prediction, output_dict=True)

report

{'FALSE': {'precision': 0.75, 'recall': 0.75, 'f1-score': 0.75, 'support': 4},
 'TRUE': {'precision': 0.5,
  'recall': 1.0,
  'f1-score': 0.6666666666666666,
  'support': 1},
 'partially false': {'precision': 0.75,
  'recall': 0.6,
  'f1-score': 0.6666666666666665,
  'support': 5},
 'accuracy': 0.7,
 'macro avg': {'precision': 0.6666666666666666,
  'recall': 0.7833333333333333,
  'f1-score': 0.6944444444444443,
  'support': 10},
 'weighted avg': {'precision': 0.725,
  'recall': 0.7,
  'f1-score': 0.7,
  'support': 10}}

In [333]:
metrics.confusion_matrix(y_test, prediction)

array([[3, 0, 1],
       [0, 1, 0],
       [1, 1, 3]])